In [ ]:
!ls ../

In [2]:
import sqlite3
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np

In [3]:
DATES_COLS = ['date_start', 'date_end']

In [ ]:

# Create your connection.
cnx = sqlite3.connect('../etl/db/db.db')

df = pd.read_sql_query("SELECT * FROM premium_payments", cnx, parse_dates=DATES_COLS)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end'])

In [ ]:
df

In [ ]:
df.member_id.value_counts()

In [ ]:
# id 1553 get differents type of premium

In [ ]:
df[df['member_id'] == 1553]

In [ ]:
sample = df[df['member_id'] == 1553]

In [ ]:
sample

In [ ]:
sample['test'] = sample.groupby(['member_id', 'type'])['date_end'].shift(1)

In [ ]:
sample

In [ ]:
sample['date_start'].min()

In [ ]:
#df.merge(df.groupby(['C', 'D']).apply(lambda row: row['B'] - row['A']).sum(level=[0,1]).reset_index())

In [ ]:
sample.groupby(['member_id', 'type']).apply(lambda row: row['date_end'].max()-row['date_start'].min())

In [ ]:
sample.groupby(['member_id', 'type']).transform(lambda row: row['date_end'].max()-row['date_start'].min())

In [ ]:
result = sample.groupby(['member_id', 'type']).apply(lambda row: row['date_end'].max()-row['date_start'].min())

In [ ]:
result

In [ ]:
sample['relativedelta'] = 

In [ ]:
sample['transaction_class'] = np.where(sample['date_start']+pd.Timedelta(3, unit='D')<sample['test'], 'other_transaction', 'same transaction')

In [ ]:
sample

In [ ]:
sample.groupby(['member_id', 'type', 'transaction_class']).agg({
    'date_start': min,
    'date_end': max
})

In [ ]:
sample['test_bis'] = sample['date_start'] - sample['test']

In [ ]:
sample

In [ ]:
sample['test_bis'] + pd.Timedelta(3, unit='D') 

In [ ]:
experiment = df.copy()

In [ ]:
# get number of members
df.member_id.nunique()

In [ ]:
def _group_transaction(df):
    df['end_date_alg'] = df.groupby(['member_id', 'type'])['date_end'].shift(1)
    return np.where(df['date_start']+pd.Timedelta(3, unit='D')<df['end_date_alg'], 'other_transaction', 'same transaction')

In [ ]:
df['date_start']+pd.Timedelta(3, unit='D')

In [ ]:
experiment['transaction_class'] = _group_transaction(experiment)

In [ ]:
experiment

In [ ]:
experiment.transaction_class.value_counts()

In [ ]:
result = experiment.groupby(['member_id', 'type', 'transaction_class']).agg({
    'date_start': min,
    'date_end': max
})

In [ ]:
result

In [ ]:
result.columns

In [ ]:
result.index

In [ ]:
result.reset_index(level='transaction_class').drop('transaction_class', axis=1)

In [ ]:
final_result = result.reset_index().drop('transaction_class', axis=1)

In [ ]:
final_result

In [ ]:
final_result.member_id.value_counts()

In [ ]:
final_result[final_result['member_id']==9637]

check results 

In [ ]:
!ls ../etl/db

In [ ]:
# Create your connection.
cnx = sqlite3.connect('../etl/db/db_transformed.db')

df = pd.read_sql_query("SELECT * FROM premium_payments_transformed", cnx)

In [ ]:
df

In [17]:
# Create your connection.
cnx = sqlite3.connect('../etl/db/db_transformed.sqlite3')

df_sqlite = pd.read_sql_query("SELECT * FROM premium_payments_transformed", cnx)

In [18]:
df_sqlite

,member_id,type,date_start,date_end
0,1,1,2018-10-31 00:00:00,2019-01-01 00:00:00
1,2,3,2019-01-18 00:00:00,2019-04-23 00:00:00
2,3,3,2019-06-28 00:00:00,2019-07-29 00:00:00
3,3,3,2019-03-30 00:00:00,2019-04-29 00:00:00
4,4,2,2020-02-14 00:00:00,2020-03-16 00:00:00
...,...,...,...,...
16993,9997,1,2019-01-07 00:00:00,2019-02-06 00:00:00
16994,9997,2,2019-02-08 00:00:00,2019-03-11 00:00:00
16995,9998,1,2019-09-03 00:00:00,2019-12-07 00:00:00
16996,9998,2,2019-06-05 00:00:00,2019-07-05 00:00:00


In [ ]:
df_sqlite.equals(df)

In [19]:
df_sqlite.member_id.value_counts()

3756    4
9919    4
714     4
332     4
4701    4
       ..
4763    1
4762    1
4754    1
4752    1
9999    1
Name: member_id, Length: 9999, dtype: int64

In [21]:
df_sqlite[df_sqlite['member_id'] == 3756]

,member_id,type,date_start,date_end
6353,3756,2,2019-04-08 00:00:00,2019-05-09 00:00:00
6354,3756,2,2019-01-21 00:00:00,2019-02-20 00:00:00
6355,3756,3,2019-07-26 00:00:00,2019-08-26 00:00:00
6356,3756,3,2019-05-10 00:00:00,2019-06-10 00:00:00


DEBUG FUNCTION

In [6]:
data = {
        'member_id': [1, 1, 1, 1, 2],
        'date_start': ['2018-12-01', '2018-10-31', '2019-01-10', '2019-02-10', '2020-01-01'],
        'date_end': ['2019-01-01', '2018-11-30', '2019-02-09', '2019-03-11', '2020-02-01'],
        'type': [1, 1, 1, 2, 3]
    }
input_df = pd.DataFrame(data)
input_df['date_start'] = pd.to_datetime(input_df['date_start'])
input_df['date_end'] = pd.to_datetime(input_df['date_end'])

In [7]:
input_df

,member_id,date_start,date_end,type
0,1,2018-12-01,2019-01-01,1
1,1,2018-10-31,2018-11-30,1
2,1,2019-01-10,2019-02-09,1
3,1,2019-02-10,2019-03-11,2
4,2,2020-01-01,2020-02-01,3


In [17]:
group_cols = ['member_id', 'type']

In [12]:
start_date_col = 'date_start'

In [20]:
group_cols + [start_date_col]

['member_id', 'type', 'date_start']

In [16]:
group_cols

['member_id', 'type', 'date_start']

In [21]:
input_df.sort_values(by=group_cols+[start_date_col])

,member_id,date_start,date_end,type
1,1,2018-10-31,2018-11-30,1
0,1,2018-12-01,2019-01-01,1
2,1,2019-01-10,2019-02-09,1
3,1,2019-02-10,2019-03-11,2
4,2,2020-01-01,2020-02-01,3


In [8]:
input_df['date_end_lag'] = input_df.groupby(['member_id', 'type'])['date_end'].shift(1)

In [9]:
input_df

,member_id,date_start,date_end,type,date_end_lag
0,1,2018-10-31,2018-11-30,1,NaT
1,1,2018-12-01,2019-01-01,1,2018-11-30
2,1,2019-01-10,2019-02-09,1,2019-01-01
3,1,2019-02-10,2019-03-11,2,NaT
4,2,2020-01-01,2020-02-01,3,NaT


In [15]:
input_df['transaction'] = np.where(input_df['date_end_lag'] + pd.Timedelta(3, unit='D') < input_df['date_start'], 'other_transaction', 'same_transaction')

In [16]:
input_df

,member_id,date_start,date_end,type,date_end_lag,transaction
0,1,2018-10-31,2018-11-30,1,NaT,same_transaction
1,1,2018-12-01,2019-01-01,1,2018-11-30,same_transaction
2,1,2019-01-10,2019-02-09,1,2019-01-01,other_transaction
3,1,2019-02-10,2019-03-11,2,NaT,same_transaction
4,2,2020-01-01,2020-02-01,3,NaT,same_transaction


In [23]:
result = input_df.groupby(['member_id', 'type', 'transaction']).agg({
                'date_start': min,
                'date_end': max
            }
            )

In [25]:
result.sort_values(by=['member_id', 'type', 'date_start'])

date_start   date_end
member_id type transaction                            
1         1    same_transaction  2018-10-31 2019-01-01
               other_transaction 2019-01-10 2019-02-09
          2    same_transaction  2019-02-10 2019-03-11
2         3    same_transaction  2020-01-01 2020-02-01

-> add sort 